# Creating Magnifier Inset Effect with `ggbunch()`
The `ggbunch()` here combines two plots into a single figure with custom layout - a main scatter plot showing diamond price vs. carat weight colored by cut quality, and a zoomed-in inset that magnifies a specific region of interest.

In [ ]:
%useLatestDescriptors
%use dataframe
%use lets-plot

In [ ]:
LetsPlot.getInfo()

In [ ]:
val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/refs/heads/master/data/diamonds.csv")
print(df.size())
df.head(3)

In [ ]:
// Convert the 'cut' column to categorical data type with specified order
val cutCategories = listOf("Fair", "Good", "Very Good", "Premium", "Ideal")

val pBase = (letsPlot(df.toMap()) { x = "carat"; y = "price"
                                    color = asDiscrete("cut", levels=cutCategories)
                                  }
 + geomPoint() 
 + geomSmooth(deg=2, size=1, tooltips=tooltipsNone)
 + scaleColorViridis(direction=-1)
)

In [ ]:
// The region of interest
val caratMin = 2.1
val caratMax = 2.7
val priceMin = 12650
val priceMax = 17100

In [ ]:
// The main plot
val pMain = (pBase
    + geomRect(
        xmin = caratMin,
        xmax = caratMax,
        ymin = priceMin,
        ymax = priceMax,
        alpha = 0.4, 
        inheritAes = false, 
        fill = "white"
    )
    + themeGrey()
    + coordCartesian(ylim = Pair(0, 20000))
    + theme()
        .legendPosition(0.58, 0.02)
        .legendJustification(0, 0)
)

pMain

In [ ]:
// The 'magnifier' plot
val pMag = (pBase
    + themeMinimal()
    + coordCartesian(
        xlim = Pair(caratMin, caratMax),
        ylim = Pair(priceMin, priceMax)
    )
    + themeVoid()
    + theme(plotInset = 0).legendPositionNone() 
)

pMag

In [ ]:
ggbunch(
    plots = listOf(
        pMain + geomSegment(
                x = caratMin, 
                y = priceMin, 
                xend = 4.30, 
                yend = 5800, 
                inheritAes = false, 
                size = 0.5
        ) + geomSegment(
                x = caratMin, 
                y = priceMax, 
                xend = 4.30, 
                yend = 19100, 
                inheritAes = false, 
                size = 0.5
        ),
        pMag + theme(plotBackground = elementRect(size = 2, fill = "rgba(255, 255, 255, 0.7)"))
    ),
    regions = listOf(
        listOf(0, 0, 0.9, 1),
        listOf(0.75, 0.05, 0.25, 0.6)
    )
) + 
ggsize(900, 600) +
ggtitle("Diamond Price vs. Carat Weight by Cut Quality") +
theme(plotTitle = elementText(hjust = 0.5, size = 18))
